* Для кого автозаполнение: клиенты, система, операторы, какие-то два из них, все?
* Посмотреть на содержимое, выявить уникальные
* Убрать знаки препинания
* Убрать цифры
* Выявить биграммы
* Посчитать для каждого слова количество упоминаний остальных слов после него
* Определить, сколько верхних значений берем
* Определить, как выкидывать редкие пары (тупо счетчик или что-то поинтереснее)
* Пока Vectorizer автоматически приводит всё к нижнему регистру. Мб это не всегда правильно
* Нужно отфильтровать служебные слова

Новые замечания:
* Фильтровать несуществующие слова (через speller)
* Фильтровать мат. Впрочем, он и так нечасто встречается
* Объединить в одном классе предсказание следующего слова и следующей буквы 
* Лучше, если предсказание дописывания будет учитывать контекст (хотя бы предыдущее слово)
* Оптимизировать хранение данных через дерево
* Мб ёфикация (это пока от себя)

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re
import matplotlib.pyplot as plt
%matplotlib inline
import my_speller
import imp
imp.reload(my_speller)
from autocorrect import spell as spell_eng

import pygtrie as trie

ModuleNotFoundError: No module named 'autocorrect'

Загрузка датасетов

In [2]:
df1 = pd.read_csv('sdf.csv')

In [3]:
df1.head()

,REQ_NUM,INITIATOR_TYPE,MESSAGE
0,1704120255467501,System,Вашему вопросу присвоен номер #num
1,1704120255467501,Operator,Здравствуйте !
2,1704120255467501,Client,"Я внес деньги на карту . Когда они придут , я ..."
3,1704120255467501,Operator,Вы вносили деньги на дебетовую карту или на кр...
4,1704120255467501,Client,Кредитная


In [4]:
df2 = pd.read_csv('sdf2.csv')

In [5]:
df2.head()

,CHAT_ID,CHAT_STEP,EV_STEP,MESSAGE_OPERATOR,MESSAGE_CLIENT
0,1-312KFA7K,1,1,Чем могу помочь ?,NaN
1,1-312KFA7K,1,0,"Здравствуйте , #Name !",NaN
2,1-312L9W2Y,1,0,"Здравствуйте , #Name !",NaN
3,1-312L9W2Y,1,1,Чем могу вам помочь ?,NaN
4,1-312L9W2Y,2,0,Оба счета открыты в пределах одного региона ?,"Здравствуйте , какой процент удерживается в ка..."


In [8]:
df2[df2.MESSAGE_CLIENT.str.startswith('Другу').fillna(value=False)]

,CHAT_ID,CHAT_STEP,EV_STEP,MESSAGE_OPERATOR,MESSAGE_CLIENT
100828,1-32X3HBOF,4,1,"#Name , вы зашли в Сбербанк Онлайн ?",Другу
167106,1-343ZW52P,5,1,NaN,Другую
302640,1-362PA8L3,2,1,NaN,Другую
337554,1-365ZCR3B,3,1,Остались ли у вас вопросы ?,Другу
561553,1-39L2Y21P,2,0,В личный кабинет вы входили через приложение и...,Другу


In [24]:
df1[df1.MESSAGE.str.startswith('Света').fillna(value=False)]

,REQ_NUM,INITIATOR_TYPE,MESSAGE
2147,1704120277112101,Operator,"Света , для уточнения информации , мне понадоб..."
2152,1704120277112101,Operator,"Света , к сожалению реквизитов благотворительн..."
2507,1704120279219901,Operator,"Света , здравствуйте !"
2510,1704120279219901,Operator,"Света , к сожалению с #date прекращен выпуск в..."
2910,1704120280748701,Operator,"Света , выпуск карт сейчас возможен только в о..."
2912,1704120280748701,Operator,"Света , рекомендую оформить классическую карту..."
3026,1704120281314501,Operator,"Света , у вас остались вопросы ?"
3147,1704120281774501,Operator,"Света , возможно вас заинтересует вклад , Сохр..."
3156,1704120281793701,Operator,"Света , что для вас в приоритете : снимать , п..."
3162,1704120281793701,Operator,"Света , вы можете открыть вклад с повышенными ..."


In [23]:
df2[df2.MESSAGE_CLIENT.str.startswith('Света').fillna(value=False)]

,CHAT_ID,CHAT_STEP,EV_STEP,MESSAGE_OPERATOR,MESSAGE_CLIENT


In [ ]:
df1[df1.REQ_NUM == 1706240246262701]['MESSAGE'].values

In [25]:
df1[df1.REQ_NUM == 1705080673946501]

,REQ_NUM,INITIATOR_TYPE,MESSAGE
100768,1705080673946501,System,Вашему вопросу присвоен номер #num
100769,1705080673946501,Client,"Света , расскажите анекдот"
100770,1705080673946501,Operator,"Здравствуйте , #Name ! \ n \ n"
100771,1705080673946501,Operator,Чем могу вам помочь ?
100772,1705080673946501,Client,Или смешную историю


In [16]:
initiator = df1[df1.REQ_NUM == 1706240246262701]['INITIATOR_TYPE'].values

In [17]:
d = {'Operator':'(О) ', 'Client':'(К) '}
for m, i in zip(mes, initiator):
    print('- ', d[i], m)

-  (О)  Здравствуйте , #Name ! Чем могу помочь ?
-  (К)  Привет
-  (К)  Мне нужен кредит
-  (О)  Вас интересует потребительский или жилищный кредит ?
-  (К)  Ну как сказать
-  (О)  На какие цели планируете взять кредит ?
-  (К)  Другу на день рождения снять квартиру и шлюх
-  (К)  Что то совместное наверное
-  (О)  Вы можете рассмотреть предложения Сбербанка на выдачу потребительских кредитов .
-  (К)  Я плохо вижу
-  (О)  Уточнить условия и подобрать удобный для вас кредит вы можете , пройдя по ссылке : #url
-  (К)  Как вы считаете , как кредит мне больше подходит ?
-  (К)  Какой
-  (О)  Вы можете подать заявку на потребительский кредит без обеспечения . Условия кредитования вы можете уточнить , пройдя по ссылке : #url
-  (К)  В чем его преимущества в отличии от других ?
-  (О)  Данный вид кредита не является целевым , минимальные сроки рассмотрения заявки , удобная форма подачи заявки и сумма кредита может быть от #num рублей и до #num рублей .
-  (К)  Мне 25 надо
-  (О)  Прошу проще

Создадим векторизатор. Чтобы включить в статистику слова из одной буквы, нужно переопределить регулярное выражение для поиска слов

In [2]:
CV = CountVectorizer(ngram_range=(2,2), token_pattern='\w{2,}|[aивжкосуя]', stop_words=['amp', 'quo', 'nbsp', 'raquo', 'laquo'])

Модуль исправления ошибок

In [27]:
fn_dict = 'data/word_freq_dict.csv.zip'
fn_letters = 'data/letter_typos_matrix.json'
fn_typical = 'data/typical_typos.json'
fn_bigrams = 'data/bigrams.zip'
speller = my_speller.Speller(fn_dict, fn_letters, fn_typical, fn_bigrams, fix_register=True, verbose=False, try_split=True)

Перед запуском векторизатора удаляем NaN, цифры и нижние подчеркивания, исправляем опечатки

*Cпеллчекер работает весьма медленно. Возможно, стоит применить его уже после удаления всех редких пар. Тогда нужно будет применить drop_duplicates с параметром subset*

In [8]:
%%time
client_mess = pd.concat([df1[df1.INITIATOR_TYPE=='Client'].MESSAGE, df2.MESSAGE_CLIENT])
regex = re.compile('\d+|_+')
client_mess = client_mess.dropna().replace(regex, ' ').apply(speller)


Wall time: 7min 11s


In [30]:
client_mess.head()

2     Я внес деньги на карту. Когда они придут, я их...
4                                             Кредитная
6                                              По карте
8                                                    Да
10                                                   Да
dtype: object

In [32]:
client_mess.to_csv('Client_messages_cleaned.csv', encoding='Windows-1251', sep=';', header=False)

In [3]:
client_mess = pd.read_csv('Client_messages_cleaned.csv', encoding='Windows-1251', sep=';', header=None)
del client_mess[0]

In [4]:
client_mess.head(20)

,1
0,"Я внес деньги на карту. Когда они придут, я их..."
1,Кредитная
2,По карте
3,Да
4,Да
5,"ООО, вас чат появился"
6,Прикольное
7,А баланс можете сказать
8,"А нет, не надо"
9,А до # Name вы работаете


Удалим служебные слова (начинаются с решетки, бывают пробелы между решеткой и словом). Слово #num удалять не надо, так как после него будут предлагаться слова, относящиеся к цифрам.

In [5]:
client_mess.dropna(inplace=True)

In [11]:
s = '#Name saastea #num # num seedshg #Num # Num  # patronymic #name'
regex = '#\s*(?!num)\w+'
regex2 = '#\s*\w+'
re.findall(regex, s)

['#Name', '#Num', '# Num', '# patronymic', '#name']

In [12]:
client_mess[1] = client_mess[1].apply(lambda x: re.sub('#\s*(?!num)\w+', ' ', x))

In [162]:
for i in client_mess.index.values:
    #x = re.search('(\#\w+)', client_mess.loc[i, 1])
    x = re.search('amp', client_mess.loc[i, 1])
    if x:
        print(client_mess.loc[i, 1])

Меня интересует то что вы сказали & amp, quo, переводи с другой карты & amp, quo, .
Что такое пакет услуг & amp, quo, Сбербанк Премьер & amp, quo, ?
Получается со счёта & amp, quo, Пополняй & amp, quo, деньги списать не возможно ?
вопрос возник потому, что в описании вклада & amp, quo, сохраняй онлайн & amp, quo, особо указано что он доступен только лицам за лет
Цитата : Открытие Вкладов Он @ он на специальных условиях для пенсионеров в системе & amp, laquo, Сбербанк Он @ он & amp, raquo, , банкоматах, платежных терминалах Сбербанка осуществляется для клиентов старше лет .
Вы видели фильм & amp, quo, а зори здесь тихие & amp, quo ,
Вклад & amp, quo, Управляй & amp, quo, в рублях. Проценты будут взиматься, если в другом городе снимать наличку - вот главный вопрос !
Здравствуйте, вклад & amp, quo, Память поколений & amp, quo, возможно оформить только на три месяца ?
Здравствуйте. Посчитайте, пожалуйста, сколько я получу по вкладу & amp, quo, Память поколений & amp, quo, с вложенных   руб

Какова стоимость опции & amp, laquo, Быстрый платёж & amp, raquo, ,, в рублях, предоставляемой в рамках услуги, , & amp, laquo, Мобильный банк & amp, raquo, ?
Какая комиссия в рублях взимается за, отмену перевода & amp, quo, Колибри & amp, quo, ?
Что значит & amp, quo, с учетом перехода через пенсионный возраст & amp, quo, и какая разница в выплатах по кредиту .
Даже зашла во вкладку & amp, quo, Помощь & amp, quo, в личном кабинете, посмотрела там тоже написано, что должна быть вкладка & amp, quo, взять кредит & amp, quo, или & amp, quo, оформить онлайн заявку & amp, quo,. и нет ничего .
& amp, quo, Добрый день ! У меня есть материнский капитал. Как производится погашение уже выданного жилищного кредита или первоначального взноса ?, & amp, quo ,
Добрый день, в течение   минут рабочая сессия Сбол останется активной, если я не буду совершать активных операций ? & amp, nbsp, & amp, nbsp ,
У меня есть карта & amp, quo, Маэстро & amp, quo, - пенсионная. Могу оформить онлайн кредит на эту ка

Добрый день ! Подскажите, пожалуйста : если я положу деньги на вклад & amp, quo, Сберегательный счет & amp, quo, в одном городе, могу ли я их свободно снять в другом городе ?
Добрый день, возможно ли списание баллов & amp, quo, спасибо & amp, quo, при оплате с помощью сервиса Samsung Pay ?
Была пару лент назад карта, выпускал в Подмосковье. я заказал карту & amp, quo, Молодежная & amp, quo,  
Город Вичуга, Ивановская область. Когда стоит ждать пенсию на карту вашего & amp, quo, скоростного & amp, quo, банка ? Или вы нас к скоту уже приравняли ?
июня я забрал из банка свою кредитную карточку ( на счету было.   ). Потратил. Внёс обратно ( ровно.   ) примерно в конце июня. Сейчас июля, срок возврата денег на счет начнется с нуля если я снова начну ею пользоваться ? Или нужно было как то & amp, quo, закрывать & amp, quo, предыдущий кредит ?
& amp, quo, закрывать & amp, quo ,
хотел поинтересоваться по поводу & amp, quo, больше Сбербанк онлайн & amp, quo, , а в частности стоимости подключени

МУП Много РК & amp quo Жилсервискерчь & amp quo   :  
Здравствуйте. У меня подключена услуга & amp quo МОБ. банк & amp quo. Но сообщения перестали приходить .
 , здравствуйте ! Мне одобрили потребительский кредит в & amp quo Сбербанк-онлайн & amp quo. Сколько у меня есть времени для принятия решения в получении кредита ?
Здравствуйте ! У меня в сентябре заканчивается срок действия карты & amp quo Маэстро & amp quo. Лежу на излечение. Как мне получить и привязать новую карту & amp quo Маэстро & amp quo ?
Здравствуйте ! Хочу узнать что нужно, чтобы получить кредитную карту & amp quo Молодёжная & amp quo ? Какие нужны справки и документы ?
Здравствуйте ! Хочу узнать как получить кредитную карту & amp quo Молодёжная & amp quo ? Какие справки и документы нужны для оформления ?
  Здравствуйте ! Хочу узнать как получить кредитную карту & amp amp amp quo Молодёжная & amp amp amp quo ? Какие справки и документы нужны для оформления ?
Был в СМС пароль, для чего он я так и не поняла, так как в & 

Добрый день, можно ли в рамках кредита & amp quo Кредит на покупку готового жилья & amp quo приобрести частный дом под залог квартиры ( вторична ) ?
Тоесть отказаться от этой и перезаказать ту которую я хочу ? И еще, что за & amp quo шумиха & amp quo с новыми картами на которые будет переходить Россия ?
Итак вопрос : Как мне подтвердить свое авторское право на изображение для индивидуального дизайна карты в кратчайшие сроки и не через сайт ? ( т. к. запрос стоит в обработки а до сентября, а я уже улетаю из города в это время и мне нужна карта ) Изначальная проблема - не принят дизайн индивидуальной карты, по всем признакам потому, что картинку посчитали & amp quo украденной & amp quo
Вот, наконец то компетентный человек. Значит мне нужно прийти в офис, указанный в заявке как & amp quo где получить карту & amp quo и лично переговорить с дизайнером, предоставив доказательства своих прав на   ?
Извиняюсь, не нашёл определения & amp quo Молодая семья & amp quo
Здравствуйте как подключить о

In [ ]:
stopwords = ['amp', 'quo', 'nbsp', 'raquo', 'laquo']

Запустим CountVectorizer на клиентских сообщениях.

In [13]:
%%time
cnt = CV.fit_transform(client_mess[1])

Wall time: 28.1 s


In [14]:
cnt.shape

(812312, 833289)

Выделим все биграммы и подсчитаем суммарное количество упоминаний во всех сообщениях

In [15]:
%%time
# Все биграммы
features = CV.get_feature_names()
# Суммарное упоминание каждой биграммы во всех сообщениях
total_count = np.array(np.sum(cnt, axis=0))

Wall time: 1.54 s


In [16]:
%%time
# Поделим биграммы на первое и второе слово
first_word = [x.split(' ')[0] for x in features]
second_word = [x.split(' ')[1] for x in features]

Wall time: 1.22 s


Датафрейм с парами слов и количеством появлений

In [17]:
w = pd.DataFrame({'first_word':first_word, 'second_word':second_word, 'count':total_count.reshape(-1)})
w.to_csv('All_bigrams.csv', encoding='Windows-1251')

In [18]:
w.head()

,count,first_word,second_word
0,8,a,a
1,1,a,accout
2,1,a,applepay
3,2,a,applicatio
4,2,a,arte


In [21]:
w[w.first_word == 'num'].sort_values(['count'], ascending=False)

,count,first_word,second_word
10061,5399,num,рублей
10052,4164,num,руб
9035,3461,num,на
10454,2564,num,тысяч
10451,2498,num,тыс
7824,1483,num,num
8633,1168,num,и
8076,1149,num,в
8270,1005,num,года
8282,687,num,году


Отсортируем датафрейм по первому слову в группе и количеству появлений. Далее сгруппируем его по первому слову в биграмме. Самые популярные можно отфильтровать, если пронумеровать каждый элемент в группе по порядку. Это делается функцией cumcount

In [22]:
G = w.sort_values(['first_word', 'count'], ascending=False).groupby(['first_word']).cumcount()

Объединим исходную таблицу и столбец с номерами строки в группе

In [23]:
G1 = pd.concat([w, G], axis=1)

In [24]:
G1.sort_values(['first_word', 'count'], ascending=False, inplace=True)

In [25]:
G1[G1.first_word=='num']

,count,first_word,second_word,0
10061,5399,num,рублей,0
10052,4164,num,руб,1
9035,3461,num,на,2
10454,2564,num,тысяч,3
10451,2498,num,тыс,4
7824,1483,num,num,5
8633,1168,num,и,6
8076,1149,num,в,7
8270,1005,num,года,8
8282,687,num,году,9


In [26]:
G1.to_csv('word_pairs.csv')

Убираем слова, неизвестные спеллчекеру

In [28]:
G1['is_known'] = G1['first_word'].apply(speller._word_is_known)

In [29]:
G1['is_known2'] = G1['second_word'].apply(speller._word_is_known)

In [30]:
G1.dropna(inplace=True)

In [31]:
G1.to_csv('word_pairs_with_known_words.csv', encoding='Windows-1251')

In [32]:
G1[G1.first_word == 'num']

,count,first_word,second_word,0,is_known,is_known2
10061,5399,num,рублей,0,num,рублей
10052,4164,num,руб,1,num,руб
9035,3461,num,на,2,num,на
10454,2564,num,тысяч,3,num,тысяч
10451,2498,num,тыс,4,num,тыс
7824,1483,num,num,5,num,num
8633,1168,num,и,6,num,и
8076,1149,num,в,7,num,в
8270,1005,num,года,8,num,года
8282,687,num,году,9,num,году


Убираем редкие биграммы и оставляем для каждого слова не более трех популярных вариантов

In [33]:
G1 = pd.read_csv('word_pairs_with_known_words.csv', encoding='Windows-1251')
G1.head()

,Unnamed: 0,count,first_word,second_word,0,is_known,is_known2
0,833287,1,ёще,раз,0,ёще,раз
1,833282,1,ёж,выполнен,0,ёж,выполнен
2,833283,1,ёж,платежа,1,ёж,платежа
3,833279,1,ящичек,около,0,ящичек,около
4,833278,1,ящиком,может,0,ящиком,может


In [35]:
min_count = 40
max_variants = 20
G1_filt = G1[(G1['count'] > min_count) & (G1['0'] < max_variants)]

In [36]:
G1_filt.to_csv('word_pairs__20variants.csv', encoding='Windows-1251')

In [37]:
bigrams = G1_filt[["first_word","second_word", "count"]]
bigrams.columns = ["word1", "word2", "num"]
bigrams.index = range(len(bigrams))

In [40]:
bigrams = bigrams[bigrams.word2 != 'num']

In [41]:
bigrams.head(20)

,word1,word2,num
0,ясно,спасибо,2168
1,яндекс,деньги,92
2,являюсь,клиентом,826
3,являюсь,зарплатным,295
4,являюсь,вашим,224
5,являюсь,гражданином,215
6,являюсь,держателем,184
7,являюсь,получателем,127
8,являюсь,участником,69
9,являюсь,владельцем,66


In [42]:
bigrams.to_csv('bigrams.csv', encoding='Windows-1251', index=False)

In [46]:
bigrams[bigrams.num < 50]

,word1,word2,num
12,являюсь,пенсионером,47
13,являюсь,гражданкой,42
14,являюсь,сотрудником,41
37,юридическое,лицо,48
50,эту,программу,46
64,этот,момент,46
65,этот,процесс,43
66,этот,период,41
106,этого,нет,44
107,этого,вклада,43


In [145]:
def get_bigrams_stat(df):
    bigrams = dict()
#    total_word_entries = df.num.sum()
    for w1, w2, n in zip(df.word1, df.word2, df.num):
        bigrams[w1] = bigrams.get(w1, dict())
        bigrams[w1][w2] = n #/ total_word_entries
    return bigrams

In [146]:
bigrams_d = get_bigrams_stat(bigrams)

In [8]:
G1_filt

,Unnamed: 0,count,first_word,second_word,0,is_known,is_known2
278,911053,2167,ясно,спасибо,0,ясно,спасибо
785,910504,92,яндекс,деньги,0,яндекс,деньги
1489,909653,825,являюсь,клиентом,0,являюсь,клиентом
1490,909626,296,являюсь,зарплатным,1,являюсь,зарплатным
1491,909535,224,являюсь,вашим,2,являюсь,вашим
1492,909574,215,являюсь,гражданином,3,являюсь,гражданином
1493,909591,185,являюсь,держателем,4,являюсь,держателем
1494,909734,127,являюсь,получателем,5,являюсь,получателем
1495,909800,69,являюсь,участником,6,являюсь,участником
1496,909543,66,являюсь,владельцем,7,являюсь,владельцем


In [112]:
G1_filt[G1_filt.first_word == 'мудаки'].second_word

Series([], Name: second_word, dtype: object)

In [148]:
stopword = 'хуй'
G1_filt[(G1_filt.first_word == stopword) | (.second_word == stopword)]#['count'].sum()

,count,first_word,second_word
12429,1,name,хуй
63893,1,будешь,хуй
122443,1,вы,хуй
167968,1,деньги,хуй
221212,1,за,хуй
258385,1,идти,хуй
319445,1,кнопку,хуй
372854,1,меня,хуй
395355,1,мой,хуй
409599,13,на,хуй


In [151]:
stopword = 'сука'
G1[(G1.first_word == stopword) | (G1.second_word == stopword)]

,count,first_word,second_word,0,is_known,is_known2
842579,1,ты,сука,216,ты,сука
798769,1,сука,num,0,сука,num
798770,1,сука,глупая,1,сука,глупая
798771,1,сука,можно,2,сука,можно
798772,1,сука,нахуй,3,сука,нахуй
798773,1,сука,ты,4,сука,ты
426809,1,нахуй,сука,5,нахуй,сука
211473,1,её,сука,959,её,сука
163931,1,денег,сука,651,денег,сука
136595,2,глупая,сука,0,глупая,сука


In [153]:
df1.shape[0] + df2.shape[0]

1894157

In [157]:
stopword = 'платеж'
w[(w.first_word == stopword) | (w.second_word == stopword)]

,count,first_word,second_word
947,1,bd,платеж
2055,1,com,платеж
10918,8,name,платеж
15051,17,num,платеж
16577,1,numrub,платеж
17188,1,numрублей,платеж
20449,3,quo,платеж
21415,1,ru,платеж
23912,3,swift,платеж
25911,1,visa,платеж


In [162]:
t = trie.CharTrie()

Для каждого уникального первого слова получим суммарное число упоминаний

In [176]:
t['здравствуйте'] = True
t['здравствуй'] = True

In [177]:
t.has_subtrie('здравст')

True

In [178]:
t.values()

[True, True]

In [69]:
strings = ['Можно взять кредит в сбербанке на сумму', 'Как взять кредит в сбербанке и без страховки']
CV_s = CountVectorizer(ngram_range=(1,1))

In [70]:
cnt_s = CV_s.fit_transform(strings)

In [71]:
CV_s.get_feature_names()

['без',
 'взять',
 'как',
 'кредит',
 'можно',
 'на',
 'сбербанке',
 'страховки',
 'сумму']

In [62]:
CV_s.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (2, 2),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [76]:
re.findall('\w{2,}|[aивжкосуя]', strings[0])

['Можно', 'взять', 'кредит', 'в', 'сбербанке', 'на', 'сумму']

In [158]:
%%time
speller('долллар')

Wall time: 2 ms


'доллар'

In [45]:
re.sub('\d+', 'num', 'Можно взять кредит на рублей?')

'Можно взять кредит на рублей?'